In [7]:
import sys
import os# Navigate up one level to the parent directory and append it to sys.path
import importlib
import requests

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src import homers
import plotly.express as px
from src.config import LOGO_PATH,  BETTING_PATH

POWER_RATING_PATH = BETTING_PATH / 'power-ratings.pkl'

In [34]:
SEASON, WEEK = 2023, 9
HFA = 1.5
AVG_POINTS = 21.5

In [3]:
power_df = pd.read_clipboard()
power_df

,team,ovr,off,def
0,KC,6.5,4.1,2.4
1,PHI,6.2,4.4,1.8
2,SF,5.9,3.1,2.8
3,BAL,5.1,2.5,2.6
4,MIA,4.9,5.6,-0.7
5,DAL,4.8,2.5,2.3
6,CIN,4.7,3.2,1.5
7,BUF,4.1,3.6,0.5
8,JAX,2.7,1.8,0.9
9,DET,2.5,2.6,-0.1


In [4]:
power_df['season']=SEASON
power_df['week']=WEEK
power_df.head()

,team,ovr,off,def,season,week
0,KC,6.5,4.1,2.4,2023,9
1,PHI,6.2,4.4,1.8,2023,9
2,SF,5.9,3.1,2.8,2023,9
3,BAL,5.1,2.5,2.6,2023,9
4,MIA,4.9,5.6,-0.7,2023,9


In [8]:
power_df.to_pickle(POWER_RATING_PATH)

In [10]:
pwr = pd.read_pickle(POWER_RATING_PATH)

In [23]:
sched = nfl.import_schedules([SEASON])

In [24]:
sched = sched[sched['week']==WEEK][['game_id', 'away_team', 'home_team']].copy().set_index('game_id')
sched.head()

,away_team,home_team
game_id,,
2023_09_TEN_PIT,TEN,PIT
2023_09_MIA_KC,MIA,KC
2023_09_MIN_ATL,MIN,ATL
2023_09_SEA_BAL,SEA,BAL
2023_09_ARI_CLE,ARI,CLE


In [35]:
predict_home_score = lambda row: AVG_POINTS + row.home_off - row.away_def + HFA / 2
predict_away_score = lambda row: AVG_POINTS + row.away_off - row.home_def - HFA / 2

In [43]:
# guess the lines
gtl = pd.merge(sched, pwr[(pwr['week'] == WEEK) & (pwr['season']==SEASON)][['team', 'ovr', 'off', 'def']], left_on='away_team', right_on='team').drop(columns=['team']).rename(columns={'ovr': 'away_ovr', 'off': 'away_off', 'def': 'away_def'})
gtl = pd.merge(gtl, pwr[(pwr['week'] == WEEK) & (pwr['season']==SEASON)][['team', 'ovr', 'off', 'def']], left_on='home_team', right_on='team').drop(columns=['team']).rename(columns={'ovr': 'home_ovr', 'off': 'home_off', 'def': 'home_def'})
gtl['hfa'] = HFA
gtl['pred_away_score'] = gtl.apply(predict_away_score, axis=1)
gtl['pred_home_score'] = gtl.apply(predict_home_score, axis=1)
gtl['spread'] = gtl['pred_home_score'] - gtl['pred_away_score']
gtl['total'] = gtl['pred_home_score'] + gtl['pred_away_score']
gtl = gtl[['away_team', 'home_team', 'spread', 'total', 'hfa', 'pred_away_score', 'pred_home_score', 'away_ovr', 'home_ovr']].copy()
gtl

,away_team,home_team,spread,total,hfa,pred_away_score,pred_home_score,away_ovr,home_ovr
0,TEN,PIT,2.3,37.8,1.5,17.75,20.05,-2.3,-1.5
1,MIA,KC,3.1,51.0,1.5,23.95,27.05,4.9,6.5
2,MIN,ATL,4.0,39.3,1.5,17.65,21.65,-3.4,-0.9
3,SEA,BAL,5.2,43.3,1.5,19.05,24.25,1.4,5.1
4,ARI,CLE,7.6,36.3,1.5,14.35,21.95,-6.1,0.0
5,LA,GB,0.4,42.9,1.5,21.25,21.65,-2.3,-3.4
6,TB,HOU,2.5,41.8,1.5,19.65,22.15,-2.6,-1.6
7,WAS,NE,0.9,38.8,1.5,18.95,19.85,-2.6,-3.2
8,CHI,NO,7.7,42.6,1.5,17.45,25.15,-5.4,0.8
9,IND,CAR,-3.6,44.3,1.5,23.95,20.35,-0.5,-5.6


# Evaluate Movement between Weeks

In [3]:

previous_power = pd.read_clipboard()
previous_power

,team,ovr,off,def
0,KC,6.9,4.4,2.5
1,SF,6.5,3.4,3.1
2,PHI,5.9,4.1,1.8
3,BAL,4.7,2.1,2.6
4,DAL,4.6,2.4,2.2
5,MIA,4.5,5.7,-1.2
6,BUF,4.0,3.5,0.5
7,CIN,2.6,1.8,0.8
8,DET,2.3,2.6,-0.3
9,JAX,2.2,1.6,0.6


In [4]:
movement = pd.merge(power_df, previous_power, on='team', suffixes=('_cur', '_prev'))
for col in ['ovr', 'off', 'def']:
    movement[f'{col}_dif'] = movement[f'{col}_cur'] - movement[f'{col}_prev']
movement

,team,ovr_cur,off_cur,def_cur,ovr_prev,off_prev,def_prev,ovr_dif,off_dif,def_dif
0,KC,6.5,4.1,2.4,6.9,4.4,2.5,-0.4,-0.3,-0.1
1,PHI,6.2,4.4,1.8,5.9,4.1,1.8,0.3,0.3,0.0
2,SF,5.9,3.1,2.8,6.5,3.4,3.1,-0.6,-0.3,-0.3
3,BAL,5.1,2.5,2.6,4.7,2.1,2.6,0.4,0.4,0.0
4,MIA,4.9,5.6,-0.7,4.5,5.7,-1.2,0.4,-0.1,0.5
5,DAL,4.8,2.5,2.3,4.6,2.4,2.2,0.2,0.1,0.1
6,CIN,4.7,3.2,1.5,2.6,1.8,0.8,2.1,1.4,0.7
7,BUF,4.1,3.6,0.5,4.0,3.5,0.5,0.1,0.1,0.0
8,JAX,2.7,1.8,0.9,2.2,1.6,0.6,0.5,0.2,0.3
9,DET,2.5,2.6,-0.1,2.3,2.6,-0.3,0.2,0.0,0.2


In [6]:
# utils.download_team_pngs()

Logos downloaded to ../bin/logos


In [30]:
col = 'ovr_cur'

# Create the bar chart
fig = px.bar(movement, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(
    height=1000,
    width=800,
    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in movement.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Show the chart
fig.show()

In [39]:


fig = px.scatter(power_df, x='off', y='def', opacity=0, color='team', color_discrete_map=utils.team_unique_colors)
fig.update_layout(
    height=600,
    width=1000,

    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)
# Iterate through the data and add logos to the chart
for index, row in power_df.iterrows():
    team = row['team']
    scale = 0.9
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row['off'],  # Adjust the position
            y=row['def'],
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

fig.show()